## Carregando bibliotecas necessárias

```
pip install scikit-learn
pip install xgboost
pip install numpy
pip install pandas
```

In [29]:
import pandas as pd
import numpy as np

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from xgboost import XGBClassifier

np.random.seed(54)

## Carregando dataset final

In [30]:
PATH = './final_df.csv'
data = pd.read_csv(PATH)

def get_data_copy():
	return data.copy()

## Gerando classificação binária

In [31]:
def map_bin_classification(val, threshold):
	if val > threshold:
		return 1 
	return 0

def create_bin_classification(column, threshold):
	'Gera uma classificação binária dividindo a coluna utilizando a variável threshold'

	map_lambda = lambda x: map_bin_classification(x, threshold)
	return column.apply(map_lambda)

## Criando funções para treinar os classificadores

In [32]:
def remove_features(x,y,selector):
	'Remove features de x utilizando um seletor do sklearn'

	features = list(selector.fit(x,y).get_support(indices=True))
	x = x.iloc[:, features]

	return x

In [33]:
# colunas que não pertencem aos dados da solução do professor
REMOVE_FROM_X = ['module', 'tempo_foco', 'n_attempts']

def get_x_and_y(data, selector, y_col):
	'Divide o x e y e remove features desnecessárias'

	y = data[y_col]
	x = data.drop(columns = REMOVE_FROM_X)
	x = x.drop(columns = y_col)

	x = remove_features(x,y,VarianceThreshold(0.05))
	x = remove_features(x,y,selector)

	return x,y

In [34]:
randomForestParams = dict(
	max_depth = [5,7,11],
	n_estimators = [50,75,100]
)

xgboostParams = dict(
	n_estimators = [100, 200, 270, 300],
	learning_rate=[0.17, 0.27, 0.37],

)

def get_best_estimator(x,y,kfold,estimator, params):
	search = RandomizedSearchCV(estimator, params, n_iter=9, cv=kfold)
	search.fit(x,y)
	return clone(search.best_estimator_)

def tune_estimator(x,y,kfold,estimator):
	'Escolhe os melhores parametros para o classificador utilizando RandomizedSCV'
	if isinstance(estimator, RandomForestClassifier):
		return get_best_estimator(x,y,kfold,estimator, randomForestParams)

	if isinstance(estimator, XGBClassifier):
		return get_best_estimator(x,y,kfold,estimator,xgboostParams)

	return estimator

In [35]:
def score_estimator(x,y,estimator):
	train_x, test_x, train_y, test_y = train_test_split(x,y,stratify=y,test_size=0.33, random_state=67)
	estimator.fit(train_x, train_y)
	return estimator.score(test_x, test_y)

def train_estimator(data, estimator, selector, y_col):
	x,y = get_x_and_y(data, selector, y_col)
	tuned_estimator = tune_estimator(x,y,StratifiedKFold(n_splits=3),estimator)
	return score_estimator(x,y,tuned_estimator)


def train_estimator_by_module(data, estimator, selector, y_col):
	modules = data.module.unique()

	for module in modules:
		print(module)
		module_data = data[data.module == module]
		score = train_estimator(module_data, estimator, selector, y_col)
		print(score)

## Criando as classificações binárias de tempo e número de tentativas

O critério de divisão das classes é:

0 = abaixo ou igual à mediana

1 = acima da mediana

In [36]:
data_tempo = get_data_copy()
data_attempts = get_data_copy()

data_tempo['tempo_bin'] = create_bin_classification(data.tempo_foco, data.tempo_foco.median())
data_attempts['attempts_bin'] = create_bin_classification(data.n_attempts, data.n_attempts.median())

## Treinando os classificadores

### Treinando os classificadores com o tempo de implementação

#### Random Forest

In [37]:
train_estimator_by_module(
	data_tempo,
	RandomForestClassifier(),	
	SelectFromModel(RandomForestClassifier(n_jobs=-1), max_features=15),
	y_col="tempo_bin"
)

M01
0.9090909090909091
M02
0.7
M03
0.9411764705882353
M07
0.75
M04
0.8333333333333334
M05
0.7142857142857143
M06
1.0


#### XGBoost

In [38]:
train_estimator_by_module(
	data_tempo,
	XGBClassifier(n_jobs=-1, eval_metric="logloss", use_label_encoder=False),
	SelectFromModel(XGBClassifier(n_jobs=-1, eval_metric="logloss", use_label_encoder=False), max_features=15),
	y_col="tempo_bin"
)

M01
0.7727272727272727
M02
0.65
M03
0.8823529411764706
M07
0.75
M04
0.8333333333333334
M05
0.7857142857142857
M06
1.0


### Número de tentativas

#### RandomForest

In [39]:
train_estimator_by_module(
	data_attempts,
	RandomForestClassifier(),	
	SelectFromModel(RandomForestClassifier(n_jobs=-1), max_features=15),
	y_col="attempts_bin"
)

M01
0.6818181818181818
M02
0.75
M03
0.6470588235294118
M07
0.6875
M04
0.9166666666666666
M05
0.35714285714285715
M06
0.9


#### XGBoost

In [40]:
train_estimator_by_module(
	data_attempts,
	XGBClassifier(n_jobs=-1, eval_metric="logloss", use_label_encoder=False),
	SelectFromModel(XGBClassifier(n_jobs=-1, eval_metric="logloss", use_label_encoder=False), max_features=15),
	y_col="attempts_bin"
)

M01
0.7272727272727273
M02
0.5
M03
0.7058823529411765
M07
0.6875
M04
0.9166666666666666
M05
0.35714285714285715
M06
0.9
